In [1]:
# IMPORT STATEMENTS FOR ENTIRE PROJECT
from sympy import S, symbols, printing
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
sns.set_style('white')

In [6]:
# -----------------------------------------------
# MAKING THE LINEPLOTS WITH EXPERIMENTAL VS THEORETICAL DATA AND POLYFIT
# Add a column to dataframe with seconds instead of a timestamp
def timestamp_to_seconds(timestamp):
    totalseconds = 0
    timestamp2 = timestamp.split(":")
    totalseconds += (int(timestamp2[1]) + (int(timestamp2[0]) * 60))
    return totalseconds

# Runs conduction heat transfer equation
def conduction(roadtemp, shell_starting_temp, size):
    roadtemp += 273 # Kelvin
    t = 1 # seconds
    k = 0.1934 # W/mK
    T1 = shell_starting_temp + 273 # Kelvin
    if size.lower() == "small": 
        d = 0.004 # m
        A = (6542 * (10**-6)) * 0.642 # m^2
    elif size.lower() == "medium":
        d = 0.0050 # m
        A = (9346 * (10**-6)) * 0.590 # m^2
    return k*A*t*((roadtemp-T1)/d) # Return J transferred in that second

# q = mcat, q in Joules
def tempraised(q, size):
    if size.lower() == "small":
        m = 22.5 # g
    elif size.lower() == "medium":
        m = 36.944 # g
    c = 1.53 # j/g/C
    return q / (m*c) # Returns temperature raised (degrees)

# Uses result from tempraised to make a list of temps
def combine(roadtemp, seconds, start, size):
    temps = [start]
    for x in range(0, seconds):
        shelltemp = temps[x]
        temps.append(shelltemp + tempraised(conduction(roadtemp, shelltemp, size), size))
    return temps

# Returns average percent error over 2 lists
def error(theoretical, experimental):
    theoretical = np.array(theoretical)
    experimental = np.array(experimental)
    differences = (experimental - theoretical) / experimental
    differences = abs(differences)
    li = differences * 100
    li = pd.DataFrame(li)
    li.to_csv("%s.csv" % str(np.average(differences) * 100))
    return str(round(np.average(differences) * 100, 3))

def graphcsv(file, titletemp):
    # Read initial file with data
    data = pd.read_csv("%s" % file)
    
    # Change timestamps to seconds
    data_secondslist = []
    for x in data["Timestamp"]:
        data_secondslist.append(timestamp_to_seconds(x))
    data["Seconds"] = data_secondslist
    size = data["Size"][0]
    startingtemp = data["Starting Temp"][0]

    # Set x and y parameters
    xdata = np.array(data["Seconds"])
    ydata = np.array(data["Temperature"])

    # Get the equation fitted to a 2nd order polynomial
    p = np.polyfit(xdata, ydata, 2) # Coefficients
    f = np.poly1d(p) # The equation

    # Format equation for display
    # Calculate new x and y
    x_new = np.linspace(xdata[0], xdata[-1], max(xdata)+1)
    y_new = f(x_new)
    x = symbols("x")
    poly = sum(S("{:6.6f}".format(v))*x**i for i, v in enumerate(p[::-1]))
    eq_latex = printing.latex(poly)

    # Get theoretical data from the x and y data 
    theoretical = combine(startingtemp, max(xdata), min(y_new), size)

    # Plot the experimental data
    p1 = sns.lineplot(xdata, ydata, color='cornflowerblue', label="Experimental")
    # Plot the best fit curve
    sns.lineplot(x_new, y_new, label="${}$".format(eq_latex), color='forestgreen')
    
    # Plot the theoretical data
    sns.lineplot(np.arange(len(theoretical)), theoretical, color='red', label="Theoretical")

    # Add text with the percent error
    p1.text(max(x_new) - 150, max(ydata) - (0.25 * (max(ydata) - min(ydata))), "Avg. % Error: " + error(theoretical, y_new), 
            horizontalalignment='left', size='medium', color='black')
    
    print(error(theoretical, y_new))
    
    plt.legend(fontsize="small")
    plt.xlabel("Time Elapsed (seconds)")
    plt.ylabel("Internal Temperature (°C)")
    plt.title("%s, %s°C" % (size, titletemp))
    plt.tight_layout
    sns.despine()
    plt.savefig("./graphs/%s" % file[:-4], dpi=600)
    plt.close()

In [7]:
# Performs function on every data file
for file in glob.glob("data/*.csv"):
    graphcsv(file)

2.061


# Analysis
I think the male is probably less accurate because the shell itself had lots of tissue and stuff that broke the connection between the shell and the bottom / the inside of the shell and the thermocouple